In [5]:
import src
from src.utils import load_data_template
from src.features.games import GameFeatures

In [9]:
game_feat = GameFeatures()
data = load_data_template()
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney, fillna=0)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_season_against_opponent, fillna=0, per_game=True)
data = game_feat.per_team_wrapper(data, game_feat.games_won_in_tourney_against_opponent, fillna=0, per_game=True)
data.sample(10)

,DayNum,Season,a_win,in_target,team_a,team_b,games_won_in_season_team_a,games_won_in_season_team_a_lag-1,games_won_in_season_team_a_lag-2,games_won_in_season_team_a_lag-3,...,games_won_in_season_against_opponent_team_b,games_won_in_season_against_opponent_team_b_lag-1,games_won_in_season_against_opponent_team_b_lag-2,games_won_in_season_against_opponent_team_b_lag-3,games_won_in_tourney_against_opponent_team_a_lag-1,games_won_in_tourney_against_opponent_team_a_lag-2,games_won_in_tourney_against_opponent_team_a_lag-3,games_won_in_tourney_against_opponent_team_b_lag-1,games_won_in_tourney_against_opponent_team_b_lag-2,games_won_in_tourney_against_opponent_team_b_lag-3
454,0.0,2014,0,True,1153,1181,27,22.0,24.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
